# 🧹 Clase 04: APIs y Data Wrangling
**Curso:** Data Science II: Machine Learning para la Ciencia de Datos  
**Docente:** Leandro Araque

---

## 🚀 Contexto del Rol Profesional: "La Realidad de los Datos"

¡Hola de nuevo, equipo de Datos de **TechMarket**! 👋

La semana pasada logramos extraer datos de nuestras bases SQL y de APIs externas. Parecía un éxito rotundo, pero al revisar los archivos esta mañana, el equipo de Marketing nos ha alertado de varios problemas graves:

1.  **Datos Fragmentados:** Las ventas de las sucursales "Norte" y "Sur" llegaron en archivos separados. Hay que unificarlos. (**Concatenación**)
2.  **Información Incompleta:** Al cruzar las ventas con los clientes, faltan datos. (**Merges**)
3.  **Suciedad:** La API de usuarios nos envió datos anidados (JSON complejos) y hay muchos campos vacíos (`NaN`). No podemos hacer Machine Learning con huecos. (**Wrangling & Missing Values**)

Tu misión hoy es **limpiar, unir y estructurar** estos datos para dejarlos listos para el análisis final.

¡A limpiar se ha dicho! 🧽✨

In [1]:
# 🛠️ SETUP INICIAL: Generación de Datos Fragmentados
import pandas as pd
import numpy as np
import json

# 1. Simulación: Ventas Sucursal NORTE
df_norte = pd.DataFrame({
    'id_venta': [101, 102, 103],
    'producto': ['Laptop', 'Mouse', 'Teclado'],
    'cantidad': [2, 5, 3],
    'monto': [2000, 50, 150],
    'sucursal': ['Norte'] * 3
})

# 2. Simulación: Ventas Sucursal SUR (Mismas columnas)
df_sur = pd.DataFrame({
    'id_venta': [201, 202],
    'producto': ['Monitor', 'Laptop'],
    'cantidad': [1, 1],
    'monto': [300, 1000],
    'sucursal': ['Sur'] * 2
})

# 3. Simulación: Tabla de Descuentos (Para cruzar/Merge)
df_descuentos = pd.DataFrame({
    'producto': ['Laptop', 'Monitor', 'Teclado'], # Mouse no tiene descuento
    'descuento_pct': [0.10, 0.05, 0.15]
})

print("✅ Datos de sucursales cargados en memoria.")

✅ Datos de sucursales cargados en memoria.


## 1. 🔄 Recap: ¿Dónde estamos?
En la clase anterior aprendimos a conectar fuentes (SQL/APIs). Hoy nos enfocaremos en **Pandas** como nuestra mesa de trabajo principal.

Recordemos que en la vida real, el **60-80% del tiempo** de un Data Scientist se va en limpiar y acomodar datos (Data Wrangling). Hoy aprenderás por qué.

## 2. 🧩 Uniendo Piezas: Concat y Merge

**El Problema:** Tenemos dos reportes de ventas (Norte y Sur) separados. El Gerente Comercial quiere **UN solo reporte** total. Además, quiere aplicar los descuentos correspondientes a cada producto.

**Conceptos:**
1.  **`pd.concat` (Apilar):** Pone un DataFrame encima del otro (o al lado). Útil cuando tienen las mismas columnas. (Analogía: Pegar hojas de Excel una debajo de otra).
2.  **`pd.merge` (Cruzar):** Es el equivalente al `JOIN` de SQL. Une tablas basándose en una columna común (llave).

**Sintaxis:**
* `pd.concat([df1, df2], axis=0)` (Vertical)
* `pd.merge(df_izq, df_der, on='columna_clave', how='inner/left')`

In [ ]:
#Codigo

In [ ]:
#codigo

### 🧠 Desafío 1: Tu Turno
**Consigna:** Acaba de llegar un archivo con los **Costos de Envío** por sucursal.
Debes cruzar (`merge`) tu tabla `df_consolidado` con la tabla `df_envios` para saber cuánto cuesta enviar cada producto según su origen.

**Nota:** Usa la columna `sucursal` como llave.

In [8]:
# Datos para el desafío
df_envios = pd.DataFrame({
    'sucursal': ['Norte', 'Sur'],
    'costo_envio': [15.00, 25.50]
})

# 💻 Tu código aquí:
# df_final = ...

**SOLUCION**

## 3. 🕸️ Data Wrangling: Domando JSONs complejos

**El Problema:** La API de clientes nos devolvió un JSON, pero no es una tabla plana. Tiene "diccionarios dentro de diccionarios" (ej. la dirección está anidada). Pandas no lee esto bien automáticamente.

**Concepto:**
* **JSON Anidado:** Estructuras jerárquicas.
* **`pd.json_normalize`:** Una función mágica de Pandas que "aplana" (flatten) los JSONs, convirtiendo las claves anidadas en columnas separadas (ej. `direccion.ciudad`).

**Uso Práctico:** Fundamental para trabajar con APIs de redes sociales, e-commerce o NoSQL (MongoDB).

In [23]:
# 👨‍🏫 Ejercicio: Aplanar un JSON anidado.

# Simulemos la respuesta de una API (Lista de diccionarios anidados)
datos_api_raw = [
    {
        "id": 1,
        "nombre": "Ana",
        "contacto": {
            "email": "ana@test.com",
            "telefono": "555-1234"
        },
        "historial": [101, 102] # Lista dentro de un objeto
    },
    {
        "id": 2,
        "nombre": "Beto",
        "contacto": {
            "email": None, # Dato faltante
            "telefono": "555-9999"
        },
        "historial": []
    }
]

# Intento ingenuo (Se ve mal)
print("--- DataFrame Normal (Mal) ---")
display(pd.DataFrame(datos_api_raw))


--- DataFrame Normal (Mal) ---


,id,nombre,contacto,historial
0,1,Ana,"{'email': 'ana@test.com', 'telefono': '555-1234'}","[101, 102]"
1,2,Beto,"{'email': None, 'telefono': '555-9999'}",[]


In [24]:
# Solución con json_normalize
print("\n--- DataFrame Normalizado (Bien) ---")
df_clientes = pd.json_normalize(datos_api_raw)
display(df_clientes)


--- DataFrame Normalizado (Bien) ---


,id,nombre,historial,contacto.email,contacto.telefono
0,1,Ana,"[101, 102]",ana@test.com,555-1234
1,2,Beto,[],None,555-9999


### 🧠 Desafío 2: Tu Turno
**Consigna:** Observa el `df_clientes` que acabamos de generar.
1. La columna `historial` contiene listas. Crea una nueva columna llamada `cantidad_compras` que cuente cuántos elementos hay en esa lista para cada usuario. (Tip: usa `.apply(len)`).
2. Muestra solo el nombre y la nueva columna.

In [ ]:
# 💻 Tu código aquí:

## 4. 🕵️‍♀️ Datos Ausentes (Nulls) y Agregaciones

**El Problema:** Al unir tablas o traer datos de APIs, aparecieron `NaN` (Not a Number / Null).
* En `descuento_pct`: El Mouse no tenía descuento -> `NaN`.
* En `contacto.email`: Beto no tiene email -> `None`.

**Manejo de Nulos:**
1.  **Identificar:** `.isna().sum()`
2.  **Rellenar:** `.fillna(valor)` (Ej. rellenar descuento con 0).
3.  **Eliminar:** `.dropna()`.

**Agregaciones (GroupBy en Pandas):**
Igual que en SQL, usamos `.groupby()` para sacar reportes finales.

### 🧠 Desafío Final: Limpieza de Clientes
**Consigna:** Volvamos al DataFrame `df_clientes`.
1. Detecta cuántos nulos hay en la columna `contacto.email`.
2. Rellena los emails faltantes con el texto "no_email@techmarket.com".
3. Muestra el DataFrame limpio.

In [21]:
display (df_clientes)

,id,nombre,historial,contacto.email,contacto.telefono,cantidad_compras
0,1,Ana,"[101, 102]",ana@test.com,555-1234,2
1,2,Beto,[],None,555-9999,0


In [20]:
# 💻 Tu código aquí:

## 🏁 Misión Cumplida

¡Excelente trabajo recuperando los datos! 🛡️

Hoy lograste:
1.  **Unificar** archivos dispersos (`concat`).
2.  **Enriquecer** datos cruzando tablas (`merge`).
3.  **Estructurar** datos complejos de APIs (`json_normalize`).
4.  **Sanear** la base de datos eliminando huecos (`fillna`).

**Próxima Clase:** Con los datos limpios, pasaremos a la etapa creativa: **EDA y Visualización**. Aprenderemos a hacer gráficos que cuenten la historia de estos datos.